In [0]:
pip install faker

In [0]:
from faker import Faker
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
import random, time
from datetime import datetime
from pyspark.sql.functions import lit, to_timestamp
import concurrent.futures

In [0]:
data_schema = StructType([
  StructField("id", StringType(), True),
  StructField("name", StringType(), True),
  StructField("age", IntegerType(), True),
  StructField("address", StringType(), True),
  StructField("email", StringType(), True),
  StructField("country", StringType(), True),
  StructField("latitude", StringType(), True),
  StructField("longitude", StringType(), True),
  StructField("url", StringType(), True),
  StructField("comments", StringType(), True)
])

In [0]:
class GetWrite:
    def __init__(self, load_date, database_name, table_name, data_schema, load_type):
        self.load_date = load_date
        self.database_name = database_name
        self.table_name = table_name
        self.data_schema = data_schema
        self.load_type = load_type

    def create_data(self, id):
        return [[
        id,
        Faker().name(),
        random.randint(15,60),
        Faker().address(),
        Faker().email(),
        Faker().country(),
        Faker().latitude(),
        Faker().longitude(), 
        Faker().url(),
        Faker().text()]]
    
    def create_df(self, id):
        time.sleep(60)
        return spark.createDataFrame(obj.create_data(id), self.data_schema)

    def transform(self, id):
        df = obj.create_df(id)
        current_datetime = f'{datetime.now().strftime("%Y-%m-%dT%H:%M:%S")[:16]+":00"}'
        df_tranformed = df.withColumn("load_type" , lit(self.load_type))\
                          .withColumn("time_of_ingestion", lit(current_datetime))\
                          .withColumn("load_date" , to_timestamp(lit(self.load_date)))
        return df_tranformed

    def write_data(self, id):
        spark.sql(f"CREATE DATABASE IF NOT EXISTS {self.database_name}")
        df = obj.transform(id)
        df.write.mode("append").saveAsTable(f"{self.database_name}.{self.table_name}")